<a href="https://colab.research.google.com/github/fakabbir/AI-Playbook/blob/master/Custom%20NER%20Training%20using%20SpaCy%20%2B%20BERT(no%20static%20vector).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pip install -U spacy[cuda113]
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 2.1 MB 13.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install cupy-cuda113
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [ ]:
from pprint import pprint
import spacy
print("Spacy version: ",spacy.__version__)
nlp = spacy.load('en_core_web_trf')
pprint("Pipelines: \n")
pprint(nlp.pipeline)

Spacy version:  3.3.0
'Pipelines: \n'
[('transformer',
  <spacy_transformers.pipeline_component.Transformer object at 0x7f4a0e5fd670>),
 ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7f4a0e5d2520>),
 ('parser',
  <spacy.pipeline.dep_parser.DependencyParser object at 0x7f4a0e5d69d0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler object at 0x7f4a0e53adc0>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7f4a0e541780>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7f4a0e5d6cd0>)]


In [ ]:
doc = nlp("Bezo founded Amazon pvt ltd in 2009")

In [ ]:
doc._.trf_data.tensors[0].shape

(1, 12, 768)

In [ ]:
doc._.trf_data.tensors[1].shape

(1, 768)

In [ ]:
doc._.trf_data.model_output.last_hidden_state.shape

(1, 12, 768)

In [ ]:
doc._.trf_data.model_output.pooler_output.shape

(1, 768)

In [ ]:
#doc._.trf_data.model_output.pooler_output == doc._.trf_data.tensors[1] # Gives True

In [ ]:
TRAIN_DATA = [
    ("text1", {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0}})
]

In [ ]:
! curl https://raw.githubusercontent.com/UBIAI/Fine_tune_BERT_with_spacy3/main/test.tsv > test.tsv
! curl https://raw.githubusercontent.com/UBIAI/Fine_tune_BERT_with_spacy3/main/train.tsv > train.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22189  100 22189    0     0  91690      0 --:--:-- --:--:-- --:--:-- 91690
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  151k  100  151k    0     0   856k      0 --:--:-- --:--:-- --:--:--  856k


In [ ]:
for ent in doc.ents:
  print(ent, ent.label_,ent.has_vector)

Bezo PERSON False
Amazon pvt ltd ORG False
2009 DATE False


In [ ]:
!python -m spacy convert train.tsv ./ -t json -n 1 -c iob
!python -m spacy convert test.tsv ./ -t json -n 1 -c iob

ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): train.json
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): test.json


In [ ]:
ls

config.cfg    test.json   test.tsv    train.spacy
sample_data/  test.spacy  train.json  train.tsv


In [ ]:
!python -m spacy convert train.json ./ -t spacy
!python -m spacy convert test.json ./ -t spacy

✔ Generated output file (77 documents): train.spacy
✔ Generated output file (11 documents): test.spacy


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [38]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./test.spacy --output ./

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-02 07:49:17,400] [INFO] Set up nlp object from config
[2022-05-02 07:49:17,413] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 07:49:17,417] [INFO] Created vocabulary
[2022-05-02 07:49:17,419] [INFO] Finished initializing nlp object
[2022-05-02 07:49:19,741] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    103.78    1.29    0.80    3.37    0.01
  2     200        483.25   6773.51   61.02   58.75   63.47    0.61
  5     400         53.08   2546.55   61.56   68.03   56.22    0.62
  7     600         67.84   1857.32   61.38   61

In [39]:
ls

config.cfg   model-last/   test.json   test.tsv    train.spacy
model-best/  sample_data/  test.spacy  train.json  train.tsv


In [42]:

nlp = spacy.load("./model-best")
text = [
'''Have 5+ years of experience focused on applied machine learning, text analytics and Natural
Language Processing. 

I have managed and lead cloud based AI enabled solution’s
development from requirement gathering, architectural designing, managing multiple dev teams
and product lifecycle.
Strong computer science and programming background, with experience and proficiency in
development technologies.
'''
]
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
    print([(ent.text, ent.label_) for ent in doc.ents])

[('5+ years', 'EXPERIENCE'), ('applied machine learning', 'SKILLS'), ('text analytics', 'SKILLS'), ('Natural', 'SKILLS'), ('Processing', 'SKILLS'), ('lead cloud', 'SKILLS'), ('architectural designing', 'SKILLS'), ('managing multiple dev teams', 'SKILLS'), ('product lifecycle', 'SKILLS'), ('computer science', 'SKILLS')]


In [ ]:
l

In [34]:
cat config.cfg

[paths]
train = null
dev = null
vectors = "en_core_web_lg"
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
widt

In [37]:
s ='''
[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [5000,2500,2500,2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 256
depth = 8
window_size = 1
maxout_pieces = 3

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
seed = ${system.seed}
gpu_allocator = ${system.gpu_allocator}
dropout = 0.1
accumulate_gradient = 1
patience = 1600
max_epochs = 0
max_steps = 20000
eval_frequency = 200
frozen_components = []
annotating_components = []
before_to_disk = null

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
get_length = null

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001
t = 0.0

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"
progress_bar = false

[training.optimizer]
@optimizers = "Adam.v1"
beta1 = 0.9
beta2 = 0.999
L2_is_weight_decay = true
L2 = 0.01
grad_clip = 1.0
use_averages = false
eps = 0.00000001
learn_rate = 0.001

[training.score_weights]
ents_f = 1.0
ents_p = 0.0
ents_r = 0.0
ents_per_type = null

[pretraining]

[initialize]
vectors = ${paths.vectors}
init_tok2vec = ${paths.init_tok2vec}
vocab_data = null
lookups = null
before_init = null
after_init = null

[initialize.components]

[initialize.tokenizer]
'''
with open('config.cfg', 'w+') as f:
  f.write(s)